<a href="https://colab.research.google.com/github/parasharDev/LangChain_LangGrapgh/blob/main/LC_Chaining_Runnables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain[groq]

INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.7 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os

os.environ['GROQ_API_KEY'] = userdata.get('groqApiKey')

# Chaining

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.1-8b-instant", model_provider="groq")

In [4]:
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser

movie_title_template = PromptTemplate.from_template("Suggest me one title of a movie to watch in language {language} and genre {genre} and only title")

movie_title_chain = movie_title_template | llm | StrOutputParser()

movie_title_chain.invoke({"language": "English", "genre": "Action"})

'Casino Royale'

In [5]:
movie_summary_template = PromptTemplate.from_template("Write a short summary of the movie {movie_title}")

In [8]:
composed_chain = {"movie_title":movie_title_chain} | movie_summary_template | llm | StrOutputParser()

In [9]:
composed_chain.invoke({"language": "English", "genre": "Action"})

'Red Notice is a 2021 action-comedy film directed by Rawson Marshall Thurber and starring Dwayne Johnson, Ryan Reynolds, and Gal Gadot. The movie follows an FBI agent named John Hartley (played by Johnson), who is tasked with tracking down the world\'s greatest art thief, The Bishop (played by Reynolds). However, their cat-and-mouse game is complicated when they are joined by a brilliant con artist, Nolan Booth (played by Reynolds), and the seductive thief, "The Bishop\'s" real identity, Nolan\'s ex-partner, The Bishop (played by Gadot).\n\nAs the story unfolds, the true identity of the Bishop is revealed, and the movie takes a hilarious turn with unexpected twists and turns. The cast delivers an entertaining performance, making Red Notice a thrilling and comedic ride filled with action, humor, and heart.'

Types of Chaining
1. Sequential
2. Parallel
3. Conditional

# Runnables

Arbitrary Functions are called Runnables.
Custome Functions used as Runnables are called Runnable Lambdas

In [14]:
from langchain_core.runnables import RunnableLambda

print_title_step = RunnableLambda(lambda x: (print(x), x)[1])

composed_chain = {"movie_title":movie_title_chain} | print_title_step | movie_summary_template | llm | StrOutputParser()

composed_chain.invoke({"language": "English", "genre": "Comedy"})


{'movie_title': 'The Hangover'}


"The Hangover is a 2009 American comedy film directed by Todd Phillips. The movie follows Phil Wenneck (Bradley Cooper), a teacher, and his friends Stu Price (Ed Helms), a dentist, and Alan Garner (Zach Galifianakis), his eccentric brother, as they travel to Las Vegas for Stu's wedding. The night before the wedding, the friends wake up in their hotel suite with no memory of the previous night's events. They soon discover that they have no recollection of what happened, and one of their friends, Doug Billings (Justin Bartha), is missing. As they try to piece together the events of the previous night and find Doug, they uncover a series of wild and outrageous events that they had no memory of. The movie is a raunchy and hilarious exploration of the consequences of excessive partying and the bonds of friendship."

## Runnable Sequence

In [22]:
from langchain_core.runnables import RunnableSequence

composed_chain = RunnableSequence({"movie_title":movie_title_chain} | print_title_step | movie_summary_template | llm | StrOutputParser())

summary = composed_chain.invoke({"language": "English", "genre": "biography"})
print(summary)

{'movie_title': 'The Imitation Game'}
"The Imitation Game" is a 2014 historical drama film directed by Morten Tyldum. The movie is based on the life of Alan Turing, a British mathematician and computer scientist who played a crucial role in cracking the German Enigma code during World War II.

The film follows Turing (played by Benedict Cumberbatch) as he is recruited by the British government to lead a team of codebreakers at Bletchley Park. Despite his brilliant mathematical mind, Turing's eccentric and gay lifestyle makes him an outcast among his colleagues, and he faces prejudice and harassment from those around him.

As Turing works tirelessly to crack the Enigma code, he develops a machine called the Bombe, which helps to decipher the code and provide crucial intelligence to the Allies. However, Turing's obsession with cracking the code takes a toll on his mental health, and he struggles with the pressures of being a genius in a world where he is constantly marginalized.

The mov

## Runnable Parallel

In [20]:
from langchain_core.runnables import RunnableParallel

translate_hindi_chain = PromptTemplate.from_template("Translate the {summary} to hindi") | llm | StrOutputParser()
translate_spanish_chain = PromptTemplate.from_template("Translate the {summary} to spanish") | llm | StrOutputParser()
translate_runnable = RunnableParallel({
    "hindi": translate_hindi_chain,
    "spanish": translate_spanish_chain
})
translated_summary = translate_runnable.invoke({"summary": summary})

{'hindi': 'द सिम्युलेशन गेम 2014 की एक ब्रिटिश ऐतिहासिक नाटक फिल्म है, जिसका निर्देशन मॉर्टन टिल्डम ने किया है। फिल्म का आधार अलन ट्यूरिंग के जीवन पर आधारित है, जो एक ब्रिटिश गणितज्ञ, कंप्यूटर वैज्ञानिक और तर्कशास्त्रज्ञ थे जिन्होंने द्वितीय विश्व युद्ध के दौरान जर्मन एनिग्मा कोड को तोड़ने में महत्वपूर्ण भूमिका निभाई थी।\n\nफिल्म ट्यूरिंग की कहानी को दर्शाती है, जब वह ब्रिटिश सरकार द्वारा ब्लीटचली पार्क में एक टीम का नेतृत्व करने के लिए भर्ती किया जाता है और वहां उन्हें जर्मनों द्वारा उपयोग की जाने वाली एक जटिल क्रिप्टोग्राफिक सिस्टम, एनिग्मा कोड को तोड़ने के लिए एक मशीन, बॉम्ब का डिज़ाइन करना चाहिए। ट्यूरिंग, जो एक गे थे, उस समय के सामाजिक मान्यताओं के कारण अपने सच्चे स्वभाव को छुपाने के लिए मजबूर थे।\n\nट्यूरिंग ने एनिग्मा कोड को तोड़ने के लिए बॉम्ब मशीन का उपयोग किया, जो उनके सहयोगियों के विरोध और संदेह के बावजूद, अंततः कोड को तोड़ दिया, जिससे युद्ध के परिणाम में महत्वपूर्ण योगदान हुआ। \n\nफिल्म ने ट्यूरिंग के व्यक्तिगत संघर्षों को भी दर्शाया, जिसमें उनके संबंध और उनके द्वारा सामना 